In [1]:
import tensorflow as tf 
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras import models, Model
from tensorflow import keras
from tensorflow.keras import layers, Sequential, datasets, optimizers
import matplotlib.pyplot as plt
import numpy as np 
import glob
import os

In [2]:
import time
import random

In [3]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
        exit(-1)

# data

In [5]:
# Global Variables
BATCHSZ = 32
EPOCHS = 100

In [32]:
data_dir= "E:\Eric_HSI\hyper_data_preprocess\Salinas_w_size_9_num_200_for_2D"
data_root = glob.glob(data_dir + '/*')
for name in glob.glob(data_dir + '/*'):
    print(name)

E:\Eric_HSI\hyper_data_preprocess\Salinas_w_size_9_num_200_for_2D\data.npy
E:\Eric_HSI\hyper_data_preprocess\Salinas_w_size_9_num_200_for_2D\data_label.npy
E:\Eric_HSI\hyper_data_preprocess\Salinas_w_size_9_num_200_for_2D\test.npy
E:\Eric_HSI\hyper_data_preprocess\Salinas_w_size_9_num_200_for_2D\test_label.npy
E:\Eric_HSI\hyper_data_preprocess\Salinas_w_size_9_num_200_for_2D\train.npy
E:\Eric_HSI\hyper_data_preprocess\Salinas_w_size_9_num_200_for_2D\train_label.npy


In [33]:
train = np.load(data_root[4])
train_label = np.load(data_root[5])
test = np.load(data_root[2])
test_label = np.load(data_root[3])
train.shape, train_label.shape, test.shape, test_label.shape

((3200, 9, 9, 204), (3200,), (50929, 9, 9, 204), (50929,))

In [9]:
class_num = 16
im_height = 9
im_width = 9
im_channel = train.shape[3]
train_num = train.shape[0]
val_num = test.shape[0]

- 根据可用的CPU动态设置并行调用的数量， 应用于 num_parallel_calls
- AUTOTUNE = tf.data.experimental.AUTOTUNE
- prefetch(AUTOTUNE)
- 当GPU执行在当前批次执行前向或者后向传播时，我们希望CPU处理下一个批次的数据，以便于数据批次能够迅速被GPU使用。我们希望GPU被完全、时刻用于训练。我们称这种机制为消费者/生产者重叠，消费者是GPU，生产者是CPU。

# dataset顺序：

- 创建实例                             from_tensor_slices                       
- 重组（较大的buffer_size）             shuffle
- 重复                                  repeat
- 数据预处理、数据扩增，使用多线程等                  map
- 批次化                                batch
- 预取数据                             prefectch

In [11]:
train_label = tf.keras.utils.to_categorical(train_label)
test_label = tf.keras.utils.to_categorical(test_label)

# 根据可用的CPU动态设置并行调用的数量， 应用于 num_parallel_calls
AUTOTUNE = tf.data.experimental.AUTOTUNE

# load train dataset
train_dataset = tf.data.Dataset.from_tensor_slices((train, train_label))
# train_dataset = train_dataset.shuffle(buffer_size=train_num).repeat().batch(BATCHSZ).prefetch(AUTOTUNE)
train_dataset = train_dataset.shuffle(buffer_size=train_num).batch(BATCHSZ).prefetch(AUTOTUNE)

# load test dataset
val_dataset = tf.data.Dataset.from_tensor_slices((test, test_label))
# val_dataset = val_dataset.repeat().batch(BATCHSZ).prefetch(AUTOTUNE)
val_dataset = val_dataset.batch(BATCHSZ).prefetch(AUTOTUNE)

In [12]:
train_dataset, val_dataset 

(<PrefetchDataset shapes: ((None, 9, 9, 204), (None, 16)), types: (tf.float32, tf.float32)>,
 <PrefetchDataset shapes: ((None, 9, 9, 204), (None, 16)), types: (tf.float32, tf.float32)>)

# model
- 这个模型十分不好训练，我用2048的dense层，模型异常的大，精度几乎不会下降

In [13]:
def VGG(feature, im_height=224, im_width=224, class_num=1000, im_channel=204):
    # tensorflow中的tensor通道排序是NHWC
    # change
    # input_image = layers.Input(shape=(im_height, im_width, 3), dtype="float32")
    input_image = layers.Input(shape=(im_height, im_width, im_channel), dtype="float32")
    x = feature(input_image)
    x = layers.Flatten()(x)
    x = layers.Dropout(rate=0.5)(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(rate=0.5)(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dense(class_num)(x)
    output = layers.Softmax()(x)
    model = models.Model(inputs=input_image, outputs=output)
    return model

In [14]:
def features(cfg):
    feature_layers = []
    for v in cfg:
        if v == "M":
            # change
            # feature_layers.append(layers.MaxPool2D(pool_size=2, strides=2))
            feature_layers.append(layers.MaxPool2D(pool_size=2, strides=1))
        else:
            conv2d = layers.Conv2D(v, kernel_size=3, padding="SAME", activation="relu")
            feature_layers.append(conv2d)
    return Sequential(feature_layers, name="feature")

In [15]:
# 网络配置列表
cfgs = {
    'vgg11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'vgg13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'vgg16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'vgg19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

In [16]:
def vgg(model_name="vgg16", im_height=224, im_width=224, class_num=1000, im_channel=204):
    try:
        cfg = cfgs[model_name]
    except:
        print("Warning: model number {} not in cfgs dict!".format(model_name))
        exit(-1)
    model = VGG(features(cfg), im_height=im_height, im_width=im_width, class_num=class_num)
    return model

# train

In [17]:
# 实例化模型
model = vgg(model_name="vgg16", im_height=im_height, im_width=im_width, class_num=class_num, im_channel=204)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 9, 9, 204)]       0         
_________________________________________________________________
feature (Sequential)         (None, 4, 4, 512)         14830464  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dropout (Dropout)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               4194816   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)              

# using keras low level api for training

In [18]:
# create direction for saving weights
if not os.path.exists("save_weights_lowAPI"):
    os.makedirs("save_weights_lowAPI")

In [19]:
loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

In [20]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

In [21]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # dropout 层需要设置 trianing 
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [22]:
@tf.function
def test_step(images, labels):
    # dropout 层需要设置 trianing 
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [23]:
best_test_loss = float('inf')
train_step_num = train_num // BATCHSZ
val_step_num = val_num // BATCHSZ
for epoch in range(1, EPOCHS+1):
    train_loss.reset_states()        # clear history info
    train_accuracy.reset_states()    # clear history info
    test_loss.reset_states()         # clear history info
    test_accuracy.reset_states()     # clear history info

    t1 = time.perf_counter()
    for index, (images, labels) in enumerate(train_dataset):
        train_step(images, labels)
        if index+1 == train_step_num:
            break
    print(time.perf_counter()-t1)

    for index, (images, labels) in enumerate(val_dataset):
        test_step(images, labels)
        if index+1 == val_step_num:
            break

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    if test_loss.result() < best_test_loss:
        model.save_weights("./save_weights_lowAPI/myVGG.ckpt".format(epoch), save_format='tf')

5.144110599999998
Epoch 1, Loss: 2.535879611968994, Accuracy: 11.03125, Test Loss: 2.379885196685791, Test Accuracy: 11.79289722442627
2.8634209000000084
Epoch 2, Loss: 2.2259793281555176, Accuracy: 12.5625, Test Loss: 2.1895194053649902, Test Accuracy: 8.699323654174805
2.921755399999995
Epoch 3, Loss: 2.1155753135681152, Accuracy: 12.15625, Test Loss: 2.0884039402008057, Test Accuracy: 9.113764762878418
2.9216214000000065
Epoch 4, Loss: 1.8438879251480103, Accuracy: 21.875, Test Loss: 1.5178767442703247, Test Accuracy: 36.21346664428711
2.8975075999999973
Epoch 5, Loss: 1.6513177156448364, Accuracy: 27.90625, Test Loss: 1.4209561347961426, Test Accuracy: 30.82770347595215
2.8979975000000024
Epoch 6, Loss: 1.5108940601348877, Accuracy: 34.34375, Test Loss: 1.473039150238037, Test Accuracy: 23.744892120361328
2.921158900000023
Epoch 7, Loss: 1.409835934638977, Accuracy: 40.75, Test Loss: 1.1576513051986694, Test Accuracy: 51.327781677246094
2.9019185999999877
Epoch 8, Loss: 1.280627012

KeyboardInterrupt: 

# using keras high level api for training

In [44]:
class_num = 16
im_height = 9
im_width = 9
im_channel = train.shape[3]
train_num = train.shape[0]
test_num = test.shape[0]

In [ ]:
train_label = tf.keras.utils.to_categorical(train_label)
test_label = tf.keras.utils.to_categorical(test_label)

In [40]:
# 根据可用的CPU动态设置并行调用的数量， 应用于 num_parallel_calls
AUTOTUNE = tf.data.experimental.AUTOTUNE

# load train dataset
train_dataset = tf.data.Dataset.from_tensor_slices((train, train_label))
train_dataset = train_dataset.shuffle(buffer_size=train_num).repeat().batch(BATCHSZ).prefetch(AUTOTUNE)
# train_dataset = train_dataset.shuffle(buffer_size=train_num).batch(BATCHSZ).prefetch(AUTOTUNE)

# load test dataset
test_dataset = tf.data.Dataset.from_tensor_slices((test, test_label))
test_dataset = test_dataset.repeat().batch(BATCHSZ).prefetch(AUTOTUNE)
# val_dataset = val_dataset.batch(BATCHSZ).prefetch(AUTOTUNE)

In [41]:
train_dataset, test_dataset 

(<PrefetchDataset shapes: ((None, 9, 9, 204), (None, 16)), types: (tf.float32, tf.float32)>,
 <PrefetchDataset shapes: ((None, 9, 9, 204), (None, 16)), types: (tf.float32, tf.float32)>)

In [42]:
# 实例化模型
model = vgg(model_name="vgg13", im_height=im_height, im_width=im_width, class_num=class_num)
model.summary()

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 9, 9, 204)]       0         
_________________________________________________________________
feature (Sequential)         (None, 4, 4, 512)         9520768   
_________________________________________________________________
flatten_4 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 8192)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 512)               4194816   
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 512)              

In [36]:
# create direction for saving weights
if not os.path.exists("save_weights"):
    os.makedirs("save_weights")

In [45]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00005),
            #   from_logits=False 如果没有进行 softmax 处理，这里执行 True
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=["accuracy"])
# change + 
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.6, patience=10, min_lr=0.000001)
callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath='./save_weights/myVGG.h5',
                                                save_best_only=True,
                                                save_weights_only=True,
                                                monitor='val_loss')]

# tensorflow2.1 recommend to using fit
# history = model.fit(train_dataset,
#                     steps_per_epoch=train_num // BATCHSZ,
#                     epochs=EPOCHS,
#                     validation_data=val_dataset,
#                     validation_steps=val_num // BATCHSZ,
#                     callbacks=[callbacks, reduce_lr])
history = model.fit(train_dataset,
                    steps_per_epoch=train_num // BATCHSZ,
                    epochs=EPOCHS,
                    validation_data=test_dataset,
                    validation_steps=test_num // BATCHSZ,
                    callbacks=[callbacks, reduce_lr])

Epoch 1/100
100/100 [==============================] - 13s 129ms/step - loss: 2.5479 - accuracy: 0.0981 - val_loss: 2.2749 - val_accuracy: 0.1872
Epoch 2/100
100/100 [==============================] - 13s 126ms/step - loss: 2.1789 - accuracy: 0.1428 - val_loss: 1.9563 - val_accuracy: 0.1731
Epoch 3/100
100/100 [==============================] - 13s 128ms/step - loss: 1.7430 - accuracy: 0.2603 - val_loss: 1.5079 - val_accuracy: 0.3320
Epoch 4/100
100/100 [==============================] - 13s 127ms/step - loss: 1.5949 - accuracy: 0.3181 - val_loss: 1.3352 - val_accuracy: 0.3172
Epoch 5/100
100/100 [==============================] - 13s 125ms/step - loss: 1.5073 - accuracy: 0.3616 - val_loss: 1.3665 - val_accuracy: 0.4306
Epoch 6/100
100/100 [==============================] - 13s 126ms/step - loss: 1.2991 - accuracy: 0.4434 - val_loss: 1.1897 - val_accuracy: 0.4758
Epoch 7/100
100/100 [==============================] - 13s 126ms/step - loss: 1.2235 - accuracy: 0.4716 - val_loss: 1.0568 -